# PLOTTING ON MAPS

#### QUESTION

How could we have changed/improved the animated plot from week 1?

In [1]:
from IPython.display import Image
Image(url='data/lifeexp_fert.gif')  

In [ ]:
#pip install geopandas==0.8.0
#pip install folium

#### READ DATA

We need to import two types of data:
1. the values we want to plot (numerical data, ex. .csv );
2. the shape of the country/lands/veedel/area we want to visualize (shape file).

In [2]:
# read historical temperature data
data = 'data/all_country_temp_data_CLEAN.csv'
import pandas as pd
ddf = pd.read_csv(data)

In [3]:
ddf

,country,year,month,monthly_anomaly,monthly_uncertainty
0,Afghanistan,1900,1,-4.048,0.936
1,Afghanistan,1900,2,-2.086,1.135
2,Afghanistan,1900,3,1.835,0.933
3,Afghanistan,1900,4,-1.268,0.536
4,Afghanistan,1900,5,0.360,0.524
...,...,...,...,...,...
321673,Zimbabwe,2013,5,0.439,1.022
321674,Zimbabwe,2013,6,1.643,0.473
321675,Zimbabwe,2013,7,1.130,0.453
321676,Zimbabwe,2013,8,1.399,0.717


In [4]:
# read geographical data (what's a shape file?)
shapefile= 'data/ne_110m_admin_0_countries.shp'
import geopandas as gpd
gdf = gpd.read_file(shapefile)

C:\Users\pallotto\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\_vectorized.py:142: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [5]:
gdf

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,NAME_KO,NAME_NL,NAME_PL,NAME_PT,NAME_RU,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,geometry
0,Admin-0 country,1,6,Fiji,FJI,0,2,Sovereign country,Fiji,FJI,...,í¼ì§,Fiji,FidÅ¼i,Fiji,Ð¤Ð¸Ð´Ð¶Ð¸,Fiji,Fiji,Fiji,ææ¿,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Admin-0 country,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,United Republic of Tanzania,TZA,...,íìëì,Tanzania,Tanzania,TanzÃ¢nia,Ð¢Ð°Ð½Ð·Ð°Ð½Ð¸Ñ,Tanzania,Tanzanya,Tanzania,å¦æ¡å°¼äº,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Admin-0 country,1,7,Western Sahara,SAH,0,2,Indeterminate,Western Sahara,SAH,...,ìì¬íë¼,Westelijke Sahara,Sahara Zachodnia,Saara Ocidental,ÐÐ°Ð¿Ð°Ð´Ð½Ð°Ñ Ð¡Ð°Ñ Ð°ÑÐ°,VÃ¤stsahara,BatÄ± Sahra,TÃ¢y Sahara,è¥¿æåæ,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Admin-0 country,1,2,Canada,CAN,0,2,Sovereign country,Canada,CAN,...,ìºëë¤,Canada,Kanada,CanadÃ¡,ÐÐ°Ð½Ð°Ð´Ð°,Kanada,Kanada,Canada,å æ¿å¤§,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,Admin-0 country,1,2,United States of America,US1,1,2,Country,United States of America,USA,...,ë¯¸êµ­,Verenigde Staten van Amerika,Stany Zjednoczone,Estados Unidos,Ð¡Ð¾ÐµÐ´Ð¸Ð½ÑÐ½Ð½ÑÐµ Ð¨ÑÐ°ÑÑ ÐÐ¼ÐµÑÐ¸ÐºÐ¸,USA,Amerika BirleÅik Devletleri,Hoa Ká»³,ç¾å½,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Admin-0 country,1,5,Republic of Serbia,SRB,0,2,Sovereign country,Republic of Serbia,SRB,...,ì¸ë¥´ë¹ì,ServiÃ«,Serbia,SÃ©rvia,Ð¡ÐµÑÐ±Ð¸Ñ,Serbien,SÄ±rbistan,Serbia,å¡å°ç»´äº,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,Admin-0 country,1,6,Montenegro,MNE,0,2,Sovereign country,Montenegro,MNE,...,ëª¬í ë¤ê·¸ë¡,Montenegro,CzarnogÃ³ra,Montenegro,Ð§ÐµÑÐ½Ð¾Ð³Ð¾ÑÐ¸Ñ,Montenegro,KaradaÄ,Montenegro,èç¹å §å¥ç¾,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,Admin-0 country,1,6,Kosovo,KOS,0,2,Sovereign country,Kosovo,KOS,...,ì½ìë³´,Kosovo,Kosowo,Kosovo,Ð ÐµÑÐ¿ÑÐ±Ð»Ð¸ÐºÐ° ÐÐ¾ÑÐ¾Ð²Ð¾,Kosovo,Kosova,Kosovo,ç§ç´¢æ²,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,Admin-0 country,1,5,Trinidad and Tobago,TTO,0,2,Sovereign country,Trinidad and Tobago,TTO,...,í¸ë¦¬ëë¤ë í ë°ê³,Trinidad en Tobago,Trynidad i Tobago,Trinidad e Tobago,Ð¢ÑÐ¸Ð½Ð¸Ð´Ð°Ð´ Ð¸ Ð¢Ð¾Ð±Ð°Ð³Ð¾,Trinidad och Tobago,Trinidad ve Tobago,Trinidad vÃ Tobago,åééåæå·´å¥,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


#### DATASET PREPARATION

In [ ]:
# gdf is a bit too big and chaotic, let's clean it up!
gdf = gpd.read_file(shapefile)[['ADMIN', 'geometry']] # import only ADMIN and geometry columns
gdf.columns = ['country', 'geometry'] # rename the columns
ddf = ddf.groupby(['country', 'year'])[['monthly_anomaly']].mean().reset_index() #get the mean of the year

We need ONE only dataset with: country name, geometry of the contry and 'monthly anomaly'

In [6]:
# merge gdf and ddf
df = pd.merge(left=gdf, right=ddf, how='left', on='country')

KeyError: 'country'

In [7]:
# convert the data relative to year 2000 to JSON data
df_2000 = df[df['year'] == 2000] 
json_2000 = df_2000.to_json() 

NameError: name 'df' is not defined

#### GENERATE THE CHOROPLETH MAP

In [8]:
import folium
tiles = folium.Choropleth(
            geo_data=json_2000,                 # geojson string that includes the geo data (for the year 2000)
            name="2000 Data",                   # just a title
            data=df_2000,                      # dataframe that includes the data (for the year 2000)
            columns=['country','monthly_anomaly'], # names of the columns to include from the dataframe
            key_on='properties.country',        # name of JSON key within the "properties" value that contains country names
            fill_color='YlGnBu',                # play around with the rest of the aesthetic options
            nan_fill_color='black',         
            fill_opacity=0.8,
            line_opacity=0.1,
            legend_name='Monthly Temperature Anomaly', # Name of variable for the colour bar.
            highlight=True,
        )

NameError: name 'json_2000' is not defined

#### CREATE A WHITE BASELINE MAP

In [9]:
my_map = folium.Map(location=[52.54, 13.36], #coordinates of the center of the map
                zoom_start=2,
                tiles='CartoDB positron') #try out the other options!

In [10]:
my_map

#### MERGE AND VISUALIZE THE TWO MAPS

In [ ]:
tiles.add_to(my_map) #add the layer to the blank canvas / figure
my_map

In [ ]:
basemap.save('mymap.html')

#### MORE INFO

- folium: https://python-visualization.github.io/folium/
- geopandas and choropleph: https://geopandas.org/en/stable/gallery/choro_legends.html
- JSON: https://en.wikipedia.org/wiki/JSON